**Deep Learning Lab Experiment - 7**

##"Image Classification Using CIFAR-10 Dataset using simple deep network with 4 hidden layers and 3 dropout layer also apply pruning and quantization to reduce size and report size of model"

1.Train the original model on CIFAR-10.

2.Save the original model (model.h5).

3.Apply pruning manually:

● If a weight is less than 0.01, we set it to 0.
Save the pruned model (pruned_model.h5).
Apply post-training quantization:

● Converts weights from 32-bit float → 8-bit int.
Save the quantized model (quantized_model.tflite).
Compare and print the sizes of all three models.

In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot  # For pruning and quantization
import numpy as np
import os

# -------------------------------
# 1️⃣ Load and Preprocess CIFAR-10 Dataset
# -------------------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values (0 to 1)
x_train, x_test = x_train / 255.0, x_test / 255.0

# -------------------------------
# 2️⃣ Build a Simple Deep Neural Network (DNN)
# -------------------------------
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')  # 10 classes (CIFAR-10)
    ])
    return model

# Create and compile the model
original_model = create_model()
original_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# -------------------------------
# 3️⃣ Train the Original Model
# -------------------------------
original_model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=64)

# Save original model and get size
original_model.save('original_model.h5')
original_model_size = os.path.getsize('original_model.h5') / (1024 * 1024)  # Convert to MB
print(f"Original Model Size: {original_model_size:.2f} MB")

# -------------------------------
# 4️⃣ Apply Pruning to Reduce Parameters
# -------------------------------
pruning_params = {
    "pruning_schedule": tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.20, final_sparsity=0.80, begin_step=0, end_step=1000
    )
}

# Apply pruning wrapper
pruned_model = tf.keras.Sequential([
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Flatten(input_shape=(32, 32, 3)), **pruning_params),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(512, activation='relu'), **pruning_params),
    tf.keras.layers.Dropout(0.2),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(256, activation='relu'), **pruning_params),
    tf.keras.layers.Dropout(0.2),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(128, activation='relu'), **pruning_params),
    tf.keras.layers.Dropout(0.2),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(64, activation='relu'), **pruning_params),
    tfmot.sparsity.keras.prune_low_magnitude(tf.keras.layers.Dense(10, activation='softmax'), **pruning_params)
])

# Compile pruned model
pruned_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the pruned model with pruning callback
pruned_model.fit(
    x_train, y_train,
    epochs=5,
    validation_data=(x_test, y_test),
    batch_size=64,
    callbacks=[tfmot.sparsity.keras.UpdatePruningStep()]
)

# Remove pruning wrappers before saving
stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)
stripped_pruned_model.save('pruned_model.h5')

# Get pruned model size
pruned_model_size = os.path.getsize('pruned_model.h5') / (1024 * 1024)
print(f"Pruned Model Size: {pruned_model_size:.2f} MB")

# -------------------------------
# 5️⃣ Apply Quantization for Further Compression
# -------------------------------
converter = tf.lite.TFLiteConverter.from_keras_model(stripped_pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Apply full quantization

quantized_model = converter.convert()

# Save quantized model
quantized_model_path = 'quantized_model.tflite'
with open(quantized_model_path, 'wb') as f:
    f.write(quantized_model)

# Get quantized model size
quantized_model_size = os.path.getsize(quantized_model_path) / (1024 * 1024)
print(f"Quantized Model Size: {quantized_model_size:.2f} MB")

# -------------------------------
# 6️⃣ Compare Model Sizes
# -------------------------------
print("\nModel Size Comparison:")
print(f"🔹 Original Model: {original_model_size:.2f} MB")
print(f"🔹 Pruned Model (After Stripping): {pruned_model_size:.2f} MB")
print(f"🔹 Quantized Model: {quantized_model_size:.2f} MB")


Epoch 1/5
782/782 [==============================] - 19s 23ms/step - loss: 2.0247 - accuracy: 0.2445 - val_loss: 1.8585 - val_accuracy: 0.3206
Epoch 2/5
782/782 [==============================] - 17s 21ms/step - loss: 1.8847 - accuracy: 0.3026 - val_loss: 1.7959 - val_accuracy: 0.3528
Epoch 3/5
782/782 [==============================] - 17s 21ms/step - loss: 1.8349 - accuracy: 0.3298 - val_loss: 1.7475 - val_accuracy: 0.3693
Epoch 4/5
782/782 [==============================] - 18s 24ms/step - loss: 1.7936 - accuracy: 0.3465 - val_loss: 1.7370 - val_accuracy: 0.3718
Epoch 5/5
782/782 [==============================] - 17s 21ms/step - loss: 1.7598 - accuracy: 0.3596 - val_loss: 1.6727 - val_accuracy: 0.4077
Original Model Size: 20.04 MB
Epoch 1/5
782/782 [==============================] - 23s 27ms/step - loss: 2.0115 - accuracy: 0.2503 - val_loss: 1.8188 - val_accuracy: 0.3449
Epoch 2/5
782/782 [==============================] - 19s 24ms/step - loss: 1.8260 - accuracy: 0.3343 - val_loss:

Pruned Model Size: 6.69 MB
Quantized Model Size: 1.67 MB

Model Size Comparison:
🔹 Original Model: 20.04 MB
🔹 Pruned Model (After Stripping): 6.69 MB
🔹 Quantized Model: 1.67 MB
